# Mycobacterium tuberculosis

### Place in the input which gene you want to blast and retrieve the information

## Através do GenBank

In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

while True:
    try:
        x = input("Which gene are you looking for (rpoB, rrs or efpA):")
        if x not in ["rpoB", "rrs", "efpA"]: 
            raise ValueError("Inserted gene not valid!")
        break
    except ValueError as error:
        print(error)

#f = open("/home/alexandreesperana/Documents/Apontamentos/Laboratorios_Bioinformatica/TPratico/rpoB.gb")
f = open(f"{x}.gb")
info = SeqIO.read(f, "gb")

#print(info)
print(f"Gene Id: {info.id}")
print(f"Description: {info.description}")

taxonomia = ''.join(f"{m} | " for m in info.annotations["taxonomy"])
print(f"Taxonomy: {taxonomia[:-2]}")

print()
for i, ref in enumerate(info.annotations["references"]): print(f"\t-------- REFERENCE {i + 1} --------\n\n{ref}")

In [43]:
#print(info.features)

for value in info.features:
    if value.type == "CDS":
        gene_name = value.qualifiers["gene"][0]
        print(f"Gene Name: {gene_name}")
        print(value.qualifiers["db_xref"][0])
        protein_id = value.qualifiers["protein_id"][0]
        pt_product = value.qualifiers["product"][0]
        description = ''.join(f"Protein {protein_id} has the function: {pt_product}")
        protein = ''.join(value.qualifiers["translation"][0])
        print(f"{description}\n-------- Sequence --------\n{protein}\n")
        
    elif value.type == "rRNA":
        gene_name = value.qualifiers["gene"][0]
        print(f"Gene Name: {gene_name}")
        print(value.qualifiers["db_xref"][0])
        prot_obt = value.qualifiers["product"][0]
        print(f"Protein Obtained: {prot_obt}")

Gene Name: efpA
GeneID:888575
Protein NP_217362.1 has the function: MFS-type transporter EfpA
-------- Sequence --------
MTALNDTERAVRNWTAGRPHRPAPMRPPRSEETASERPSRYYPTWLPSRSFIAAVIAIGGMQLLATMDSTVAIVALPKIQNELSLSDAGRSWVITAYVLTFGGLMLLGGRLGDTIGRKRTFIVGVALFTISSVLCAVAWDEATLVIARLSQGVGSAIASPTGLALVATTFPKGPARNAATAVFAAMTAIGSVMGLVVGGALTEVSWRWAFLVNVPIGLVMIYLARTALRETNKERMKLDATGAILATLACTAAVFAFSIGPEKGWMSGITIGSGLVALAAAVAFVIVERTAENPVVPFHLFRDRNRLVTFSAILLAGGVMFSLTVCIGLYVQDILGYSALRAGVGFIPFVIAMGIGLGVSSQLVSRFSPRVLTIGGGYLLFGAMLYGSFFMHRGVPYFPNLVMPIVVGGIGIGMAVVPLTLSAIAGVGFDQIGPVSAIALMLQSLGGPLVLAVIQAVITSRTLYLGGTTGPVKFMNDVQLAALDHAYTYGLLWVAGAAIIVGGMALFIGYTPQQVAHAQEVKEAIDAGEL



## Obtenção do ficheiro xml com info do blast

In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW 

#f = open("/home/alexandreesperana/Documents/Apontamentos/Laboratorios_Bioinformatica/TPratico/rpoB.fasta")
f = open(f"{x}.fasta")
seq = SeqIO.read(f, "fasta")

result_handle = NCBIWWW.qblast("blastn", "nt", seq.format("fasta"))
save_file = open(f"{x}_blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

Which gene are you looking for (rpoB, rrs or efpA): efpA


## Análise Ficheiro xml

In [ ]:
from Bio.Blast import NCBIXML
import json

result_handle = open(f"{x}_blast.xml")
blast_records = NCBIXML.parse(result_handle)

allhits = []
#E_VALUE_THRESH = 0.05
for blast_record in blast_records:
    for alignment in blast_record.alignments: 
        for hsp in alignment.hsps:
            dict_all = {}
            dict_all["Info"] = alignment.hit_def
            dict_all["Hit"] = alignment.hit_id
            dict_all["Lenght"] = alignment.length
            dict_all["E-value"] = hsp.expect
            dict_all["Accession Number"] = alignment.accession
            allhits.append(dict_all)

print(f"Number of hits: {len(allhits)}")
print(json.dumps(allhits, indent = 3))

## Ideia de Fazer igual ao exemplo do site -> Uma matriz com as infos

In [ ]:
print(info)
print()
print(info.features[2])

information_resumed = ["Gene ID NCBI", "Accession Nr NCBI", "Locus Tag", "Nome", "Strand",
             "Uniprot ID", "Annotation Score", "Accession Nr NCBI", "Nome da proteína",
             "Nr Aminoácidos", "Localização celular", "Lista de termos do GeneOntology",
             "EC Number", "TC Number", "Descrição", "Comentários", "Classificação", "Sequência"]

info_add = []